In [1]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg
from keras import regularizers

Using TensorFlow backend.


In [2]:
print("The number of images with facemask labelled 'yes':",len(os.listdir('data/with_mask')))
print("The number of images with facemask labelled 'no':",len(os.listdir('data/without_mask')))

The number of images with facemask labelled 'yes': 690
The number of images with facemask labelled 'no': 668


In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax',kernel_regularizer=regularizers.l2(0.001))
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [7]:
TRAINING_DIR = "data"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.3)

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150),
                                                   subset = 'training')

validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150),
                                                            subset = 'validation' )
checkpoint = ModelCheckpoint('best.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

Found 951 images belonging to 2 classes.
Found 407 images belonging to 2 classes.


In [8]:
history = model.fit_generator(train_generator,
                              epochs=5,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])
# model = load_model('model-017.model')

Epoch 1/5
95/96 [============================>.] - ETA: 1s - loss: 0.8472 - acc: 0.5058WARNING:tensorflow:From C:\Users\sampr\Anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: best.model\assets
96/96 [==============================] - 146s 2s/step - loss: 0.8456 - acc: 0.5026 - val_loss: 0.6961 - val_acc: 0.5061
Epoch 2/5
96/96 [==============================] - 140s 1s/step - loss: 0.6729 - acc: 0.5825 - val_loss: 0.4990 - val_acc: 0.7740
Epoch 3/5
96/96 [==============================] - 129s 1s/step - loss: 0.5827 - acc: 0.6866 - val_loss: 0.6739 - val_acc: 0.5774
Epoch 4/5
96/96 [==============================] - 135s 1s/step - loss: 0.5940 - acc: 0.6982 - val_loss: 0.4309 - 

In [9]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [13]:
labels_dict={0:'without mask',1:'with mask'}
color_dict={0:(100,60,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()